In [1]:
import torch
import os
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import copy
import time
import pickle

import warnings
warnings.filterwarnings('ignore')


In [2]:
CONFIGS = {
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu",
    # specify ImageNet mean and standard deviation
    "IMG_MEAN": [0.485, 0.456, 0.406],
    "IMG_STD": [0.229, 0.224, 0.225],
    "INIT_LR": 1e-4,
    "NUM_EPOCHS": 30,
    "BATCH_SIZE": 16,
    # specify the loss weights
    "LABELS_PRDTYPE": 1.0,
    "LABELS_WEIGHT": 1.0,
    "LABELS_HALAL": 1.0,
    "LABELS_HEALTHY": 1.0,
    "MODEL_PATH": os.path.sep.join(["output", "detector.pth"]),
    "LE_PATH_PRDTYPE": os.path.sep.join(["output", "le_prdtype.pickle"]),
    "LE_PATH_WEIGHT": os.path.sep.join(["output", "le_weight.pickle"]),
    "LE_PATH_HALAL": os.path.sep.join(["output", "le_halal.pickle"]),
    "LE_PATH_HEALTHY": os.path.sep.join(["output", "le_healthy.pickle"]),
    "PIN_MEMORY": True if torch.cuda.is_available() else False,
    "DATA_BASE_PATH": os.path.join('..', 'rshiny', 'www', 'all_images'),
    "NEW_DATA_BASE_PATH": os.path.join('..', 'small_model', 'new_imgs'),
    "BASE_PATH": os.path.join('..'),
    "EARLY_STOPPING_PATIENCE": 5,
    'SMALL_MODEL_IMG_SIZE': 60
}

# create output folder
if not os.path.exists("output"):
    !mkdir -p {"output"}

In [3]:
prd_group_df = pd.read_csv(os.path.join(CONFIGS['BASE_PATH'], "bayesian_model/product_group.csv"))
# prd_group_df['group'] = pd.to_numeric(prd_group_df['group'], errors='coerce')
prd_group_df['group'] = prd_group_df['group'].fillna(0)
# prd_group_df['tokeep'] = (prd_group_df['group'].astype(int) >= 1) & (prd_group_df['group'].astype(int) <= 7)
prd_group_df['tokeep'] = prd_group_df['group'].astype(int) == 1
prd_group_df.head()

,filepath,label,ProductType,Weight,HalalStatus,HealthStatus,new_camera,tag,group,tokeep
0,IMG_20230428_123528_jpg.rf.5687b7b914f6d9aa98c...,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,1.0,True
1,IMG_20230428_123522_jpg.rf.204ff37f497f2dce442...,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,1.0,True
2,IMG_20230428_123708_jpg.rf.141ecd0cefaea75c0b7...,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,2.0,False
3,IMG_20230428_123521_jpg.rf.1069b402272252862ec...,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,1.0,True
4,IMG_20230428_123659_jpg.rf.5e1b6c4caabe48cf360...,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,2.0,False


In [4]:
annotations_0 = pd.read_csv("../master_list.csv")
print(annotations_0.shape)
annotations_0 = annotations_0[annotations_0['remove']!=1]
annotations_0 = pd.merge(annotations_0, prd_group_df[["filepath", "tokeep"]], how='left', on='filepath')

annotations_1 = annotations_0.loc[annotations_0["tokeep"]==True]
annotations_1 = annotations_1[annotations_1['tag'] != 'TestforMode5']
annotations_1 = annotations_1.groupby('ProductType').apply(lambda x: x.sample(n=min(len(x), 6))).reset_index(drop=True)
# add images whose prod type is not covered
a = annotations_0["ProductType"].unique().tolist()
b = annotations_1["ProductType"].unique().tolist()
missing_prdtype = [i for i in a if i not in b]
if len(missing_prdtype) != 0:
    # for tmp_prdtype in missing_prdtypemissing_prdtype:
    tmp_df = annotations_0[annotations_0['ProductType'].isin(missing_prdtype)]
    tmp_df.reset_index(drop=True, inplace=True)
    annotations_1_addon = tmp_df.groupby('ProductType').apply(lambda x: x.sample(n=min(len(x), 6))).reset_index(drop=True)
    annotations_1 = pd.concat([annotations_1, annotations_1_addon], ignore_index=True)
    annotations_1.reset_index(drop=True, inplace=True)

annotations_1['type'] = "old"
print(annotations_1.shape)

annotations_2 = annotations_0[annotations_0['tag'] == 'TestforMode5']
annotations_2.reset_index(drop=True, inplace=True)
annotations_2['type'] = "new"
print(annotations_2.shape)

# Concatenate the two dataframes vertically
annotations = pd.concat([annotations_1, annotations_2], ignore_index=True)
annotations.reset_index(drop=True, inplace=True)
print(annotations.shape)

(6850, 13)
(400, 15)
(10, 15)
(410, 15)


In [5]:
annotations['ProductType'].unique()

array(['BabyMilkPowder', 'Babyfood', 'BeehoonVermicelli',
       'BiscuitsCrackersCookies', 'BreakfastCereals', 'CannedBakedBeans',
       'CannedBeefOtherMeats', 'CannedBraisedPeanuts', 'CannedChicken',
       'CannedFruits', 'CannedMushrooms', 'CannedPacketCreamersSweet',
       'CannedPickles', 'CannedPorkLunchronMeat',
       'CannedSardinesMackerel', 'CannedSoup', 'CannedTunaDace',
       'CannedVegetarianFood', 'ChocolateMaltPowder', 'ChocolateSpread',
       'CoffeePowder', 'CoffeeTeaDrink', 'CookingCreamMilk',
       'CookingPastePowder', 'DarkSoySauce', 'DriedBeans', 'DriedFruits',
       'DriedMeatSeafood', 'DriedVegetables', 'FlavoredMilkDrink',
       'Flour', 'FruitJuiceDrink', 'HerbsSpices', 'InstantMeals',
       'InstantNoodlesMultipack', 'InstantNoodlesSingle', 'Jam', 'Kaya',
       'KetchupChilliSauce', 'LightSoySauce', 'MaternalMilkPowder',
       'MilkDrink', 'MilkPowder', 'Nuts', 'Oil', 'OtherBakingNeeds',
       'OtherCannedBeansPeasNuts', 'OtherCannedSeafood',
  

In [6]:
# Initialize lists for processed data
data, imagePaths, filenames, prdtypes = [], [], [], []

# Process each annotation entry
for idx, row in annotations.iterrows():
    filepath = row["filepath"]
    imagePath = os.path.join(CONFIGS["DATA_BASE_PATH"], filepath)
    # if row['Type'] == 'old':
    #     imagePath = os.path.join(CONFIGS["DATA_BASE_PATH"], filepath)
    # else:
    #     imagePath = os.path.join("/content", CONFIGS["NEW_DATA_BASE_PATH"], filepath)
    image = cv2.imread(imagePath)
    
    # Preprocess image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (CONFIGS['SMALL_MODEL_IMG_SIZE'], CONFIGS['SMALL_MODEL_IMG_SIZE']))

    # Append processed data to lists
    data.append(image)
    imagePaths.append(imagePath)
    # filenames.append(filepath.rsplit('.', 1)[0])
    filenames.append(filepath)
    prdtypes.append(row["ProductType"])
    # if row["ProductType"] == "Coffee":
    #     break

# Convert data to NumPy arrays for machine learning processing
labels = {
    'labels_prdtype': annotations['ProductType'],
    # 'labels_weight': annotations['Weight'],
    # 'labels_halal': annotations['HalalStatus'],
    # 'labels_healthy': annotations['HealthStatus'],
    # 'labels_total': annotations['label']
}

data = np.array(data, dtype="float32")
for label_name, label_data in labels.items():
    labels[label_name] = np.array(label_data)

# Split the data and labels into training and testing sets
split = train_test_split(data, *labels.values(), imagePaths, filenames,
                         test_size=0.2, random_state=42, stratify=labels['labels_prdtype'])
# split = train_test_split(data, prdtypes, imagePaths, filenames,
#                          test_size=0.2, random_state=42, stratify=prdtypes)

# Unpack the data split
(trainImages, testImages, *split_labels, trainPaths, testPaths, trainFilenames, testFilenames) = split

# Create label encoders and transform labels
le_prdtype = LabelEncoder()
# le_weight = LabelEncoder()
# le_halal = LabelEncoder()
# le_healthy = LabelEncoder()
# le_total = LabelEncoder()

trainLabels = {}
testLabels = {}

# Fit label encoders and transform labels
trainLabels['labels_prdtype'] = le_prdtype.fit_transform(split_labels[0])
testLabels['labels_prdtype'] = le_prdtype.transform(split_labels[1])

# trainLabels['labels_weight'] = le_weight.fit_transform(split_labels[2])
# testLabels['labels_weight'] = le_weight.transform(split_labels[3])

# trainLabels['labels_halal'] = le_halal.fit_transform(split_labels[4])
# testLabels['labels_halal'] = le_halal.transform(split_labels[5])

# trainLabels['labels_healthy'] = le_healthy.fit_transform(split_labels[6])
# testLabels['labels_healthy'] = le_healthy.transform(split_labels[7])

# trainLabels['labels_total'] = le_total.fit_transform(split_labels[8])
# testLabels['labels_total'] = le_total.transform(split_labels[9])


# Convert NumPy arrays to PyTorch tensors
trainImages, testImages = torch.tensor(trainImages), torch.tensor(testImages)
for label_name in labels.keys():
    trainLabels[label_name] = torch.tensor(trainLabels[label_name])
    testLabels[label_name] = torch.tensor(testLabels[label_name])


In [7]:
le_prdtype.classes_

array(['BabyMilkPowder', 'Babyfood', 'BeehoonVermicelli',
       'BiscuitsCrackersCookies', 'BreakfastCereals', 'CannedBakedBeans',
       'CannedBeefOtherMeats', 'CannedBraisedPeanuts', 'CannedChicken',
       'CannedFruits', 'CannedMushrooms', 'CannedPacketCreamersSweet',
       'CannedPickles', 'CannedPorkLunchronMeat',
       'CannedSardinesMackerel', 'CannedSoup', 'CannedTunaDace',
       'CannedVegetarianFood', 'ChocolateMaltPowder', 'ChocolateSpread',
       'CoffeePowder', 'CoffeeTeaDrink', 'CookingCreamMilk',
       'CookingPastePowder', 'DarkSoySauce', 'DriedBeans', 'DriedFruits',
       'DriedMeatSeafood', 'DriedVegetables', 'FlavoredMilkDrink',
       'Flour', 'FruitJuiceDrink', 'HerbsSpices', 'InstantMeals',
       'InstantNoodlesMultipack', 'InstantNoodlesSingle', 'Jam', 'Kaya',
       'KetchupChilliSauce', 'LightSoySauce', 'MaternalMilkPowder',
       'MilkDrink', 'MilkPowder', 'Nuts', 'Oil', 'OtherBakingNeeds',
       'OtherCannedBeansPeasNuts', 'OtherCannedSeafood',
  

In [8]:
"Coffee" in split_labels[0]

False

In [9]:
pd.Series(prdtypes).unique()

array(['BabyMilkPowder', 'Babyfood', 'BeehoonVermicelli',
       'BiscuitsCrackersCookies', 'BreakfastCereals', 'CannedBakedBeans',
       'CannedBeefOtherMeats', 'CannedBraisedPeanuts', 'CannedChicken',
       'CannedFruits', 'CannedMushrooms', 'CannedPacketCreamersSweet',
       'CannedPickles', 'CannedPorkLunchronMeat',
       'CannedSardinesMackerel', 'CannedSoup', 'CannedTunaDace',
       'CannedVegetarianFood', 'ChocolateMaltPowder', 'ChocolateSpread',
       'CoffeePowder', 'CoffeeTeaDrink', 'CookingCreamMilk',
       'CookingPastePowder', 'DarkSoySauce', 'DriedBeans', 'DriedFruits',
       'DriedMeatSeafood', 'DriedVegetables', 'FlavoredMilkDrink',
       'Flour', 'FruitJuiceDrink', 'HerbsSpices', 'InstantMeals',
       'InstantNoodlesMultipack', 'InstantNoodlesSingle', 'Jam', 'Kaya',
       'KetchupChilliSauce', 'LightSoySauce', 'MaternalMilkPowder',
       'MilkDrink', 'MilkPowder', 'Nuts', 'Oil', 'OtherBakingNeeds',
       'OtherCannedBeansPeasNuts', 'OtherCannedSeafood',
  

In [11]:
# Get the mapping from class name to index
class_to_idx = {cls: idx for idx, cls in enumerate(le_prdtype.classes_)}

# Specify your target class name
target_class_name = annotations_2['ProductType'].unique()[0]

# Initialize weights to 1 for all classes
weights = torch.ones(len(class_to_idx))

# Set the weight of the target class to 10
weights[class_to_idx[target_class_name]] = 10

criterion_prdtype = nn.CrossEntropyLoss(weight=weights)


In [12]:
class CustomTensorDataset(Dataset):
    # Initialize the constructor
    def __init__(self, images, labels, filenames, transforms=None):
        self.images = images
        self.labels = labels
        self.filenames = filenames
        self.transforms = transforms

    def __getitem__(self, index):
        # Grab the image, labels, and its bounding box coordinates
        image = self.images[index]
        label_prdtype = self.labels['labels_prdtype'][index]
        filename = self.filenames[index]

        # Transpose the image such that its channel dimension becomes the leading one
        image = image.permute(2, 0, 1)

        # Check to see if we have any image transformations to apply and if so, apply them
        if self.transforms:
            image = self.transforms(image)

        # Return a tuple of the images, labels, and bounding box coordinates
        return (image, label_prdtype, filename)

    def __len__(self):
        # Return the size of the dataset
        return len(self.images)

# Define normalization and augmentation transforms
normalization_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=CONFIGS['IMG_MEAN'], std=CONFIGS['IMG_STD'])
])

augmentation_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomRotation(20),
    # transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1)
])

# Combine augmentation and normalization for training
train_transforms = transforms.Compose([augmentation_transforms, normalization_transforms])
test_transforms = normalization_transforms

# Create PyTorch datasets
trainDS = CustomTensorDataset(trainImages, trainLabels, trainFilenames, transforms=train_transforms)
testDS = CustomTensorDataset(testImages, testLabels, testFilenames, transforms=test_transforms)

# Print dataset sizes
print("[INFO] total training samples: {}...".format(len(trainDS)))
print("[INFO] total test samples: {}...".format(len(testDS)))

# Calculate steps per epoch for training and validation set
trainSteps = len(trainDS) // CONFIGS['BATCH_SIZE']
valSteps = len(testDS) // CONFIGS['BATCH_SIZE']

# Create data loaders
# trainLoader = DataLoader(trainDS, batch_size=CONFIGS['BATCH_SIZE'], shuffle=True,
#                          num_workers=os.cpu_count(), pin_memory=CONFIGS['PIN_MEMORY'])
# testLoader = DataLoader(testDS, batch_size=CONFIGS['BATCH_SIZE'],
#                         num_workers=os.cpu_count(), pin_memory=CONFIGS['PIN_MEMORY'])

trainLoader = DataLoader(trainDS, batch_size=CONFIGS['BATCH_SIZE'], shuffle=True, num_workers=0, pin_memory=CONFIGS['PIN_MEMORY'])
testLoader = DataLoader(testDS, batch_size=CONFIGS['BATCH_SIZE'], num_workers=0, pin_memory=CONFIGS['PIN_MEMORY'])

[INFO] total training samples: 328...
[INFO] total test samples: 82...


In [13]:
# Define the MultiHeadResNet model
class MultiHeadResNet(nn.Module):
    def __init__(self, num_classes_prdtype):
        super(MultiHeadResNet, self).__init__()
        self.base_model = models.resnet18(pretrained=True)
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity()

        # Define custom fully connected layers for each prediction head
        self.fc_prdtype = nn.Linear(num_ftrs, num_classes_prdtype)

    def forward(self, x):
        x = self.base_model(x)
        prdtype = self.fc_prdtype(x)
        return prdtype

# Function to calculate accuracy
def calculate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    corrects = torch.sum(preds == labels.data)
    return corrects.double() / labels.size(0)

# Training and Validation Loop with Early Stopping
def train_model(model, criteria, optimizer, train_loader, test_loader, device, num_epochs=25, early_stopping_patience=10):
    criterion_prdtype= criteria
    best_val_loss = float('inf')
    best_model_wts = copy.deepcopy(model.state_dict())
    epochs_no_improve = 0

    history = {
        'train_loss': [],
        'train_acc_prdtype': [],
        'train_acc_overall': [],
        'val_loss': [],
        'val_acc_prdtype': [],
        'val_acc_overall': [],
    }

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects_prdtype = 0
            running_corrects_overall = 0
            total_samples = 0

            for inputs, label_prdtype, _ in train_loader if phase == 'train' else test_loader:
                inputs = inputs.to(device)
                label_prdtype = label_prdtype.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs_prdtype = model(inputs)
                    loss_prdtype = criterion_prdtype(outputs_prdtype, label_prdtype)
                    loss = loss_prdtype  # Total loss

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects_prdtype += calculate_accuracy(outputs_prdtype, label_prdtype) * inputs.size(0)
                correct_preds_overall = (outputs_prdtype.argmax(1) == label_prdtype)
                running_corrects_overall += correct_preds_overall.sum().item()
                total_samples += inputs.size(0)

            epoch_loss = running_loss / total_samples
            epoch_acc_prdtype = running_corrects_prdtype / total_samples
            epoch_acc_overall = running_corrects_overall / total_samples

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc_overall))

            if phase == 'val':
                if epoch_loss < best_val_loss:
                    print("new loss obtained")
                    best_val_loss = epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())
                    epochs_no_improve = 0
                else:
                    epochs_no_improve += 1
                    print(f"tmp epochs_no_improve: {epochs_no_improve}")

                if epochs_no_improve >= early_stopping_patience:
                    print("Early stopping triggered at epoch: {}".format(epoch + 1))
                    model.load_state_dict(best_model_wts)
                    return model, history
            
            print(f"epochs_no_improve: {epochs_no_improve}")

    model.load_state_dict(best_model_wts)
    return model, history

# Example usage of the function
# Assuming CONFIGS, trainLoader, testLoader, etc. are already defined
num_classes_prdtype = len(np.unique(trainLabels['labels_prdtype']))

custom_resnet_model = MultiHeadResNet(num_classes_prdtype)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
custom_resnet_model = custom_resnet_model.to(device)

# criterion_prdtype = nn.CrossEntropyLoss()
criterion_prdtype = nn.CrossEntropyLoss(weight=weights)

optimizer = optim.Adam(custom_resnet_model.parameters(), lr=CONFIGS['INIT_LR'])

criteria = criterion_prdtype

# Start time
print("Model training started...")
start_time = time.time()

model_ft, history = train_model(custom_resnet_model, criteria, optimizer, 
                                trainLoader, testLoader, device, 
                                num_epochs=CONFIGS['NUM_EPOCHS'], 
                                early_stopping_patience=CONFIGS['EARLY_STOPPING_PATIENCE'])

# End time
end_time = time.time()
print("Model training completed...")

execution_time = end_time - start_time
print(f"Time spent: {round(execution_time/60,2)} mins")

Model training started...
Epoch 1/30
train Loss: 4.2768 Acc: 0.0213
epochs_no_improve: 0
val Loss: 4.1411 Acc: 0.0488
new loss obtained
epochs_no_improve: 0
Epoch 2/30
train Loss: 3.4450 Acc: 0.0671
epochs_no_improve: 0
val Loss: 3.5249 Acc: 0.0732
new loss obtained
epochs_no_improve: 0
Epoch 3/30
train Loss: 2.9297 Acc: 0.1585
epochs_no_improve: 0
val Loss: 3.2825 Acc: 0.1585
new loss obtained
epochs_no_improve: 0
Epoch 4/30
train Loss: 2.6465 Acc: 0.2561
epochs_no_improve: 0
val Loss: 3.0435 Acc: 0.1951
new loss obtained
epochs_no_improve: 0
Epoch 5/30
train Loss: 2.5406 Acc: 0.2927
epochs_no_improve: 0
val Loss: 2.8684 Acc: 0.2317
new loss obtained
epochs_no_improve: 0
Epoch 6/30
train Loss: 2.2575 Acc: 0.3933
epochs_no_improve: 0
val Loss: 2.7588 Acc: 0.2927
new loss obtained
epochs_no_improve: 0
Epoch 7/30
train Loss: 1.9424 Acc: 0.5030
epochs_no_improve: 0
val Loss: 2.6489 Acc: 0.3415
new loss obtained
epochs_no_improve: 0
Epoch 8/30
train Loss: 1.7352 Acc: 0.5671
epochs_no_impro

In [14]:
torch.save(model_ft.state_dict(), 'output/multi_head_model.pth')

print("[INFO] saving label encoder...")
f = open(CONFIGS["LE_PATH_PRDTYPE"], "wb")
f.write(pickle.dumps(le_prdtype))
f.close()
# f = open(CONFIGS["LE_PATH_WEIGHT"], "wb")
# f.write(pickle.dumps(le_weight))
# f.close()
# f = open(CONFIGS["LE_PATH_HALAL"], "wb")
# f.write(pickle.dumps(le_halal))
# f.close()
# f = open(CONFIGS["LE_PATH_HEALTHY"], "wb")
# f.write(pickle.dumps(le_healthy))
# f.close()

[INFO] saving label encoder...


In [15]:
def evaluate_model(model, data_loader, dataset_size, num_mc_samples=50):
    model.eval()  # Set the model to evaluation mode
    correct_counts = {'ProductType': 0}

    with torch.no_grad():
        for (images, labels_prdtype, filenames) in data_loader:
            images = images.to(CONFIGS['DEVICE'])
            labels_prdtype = labels_prdtype.to(CONFIGS['DEVICE'])

            # Forward pass
            out1 = model(images)

            # Store deterministic predictions
            det_pred_prdtype = out1.argmax(1)

            # Update correct counts for each category
            correct_counts['ProductType'] += (out1.argmax(1) == labels_prdtype).float().sum().item()

    # Calculate accuracies
    accuracies = {key: correct_counts[key] / dataset_size for key in correct_counts}

    return accuracies

# Evaluate on training set
train_accuracies = evaluate_model(model_ft, trainLoader, len(trainDS))
print(f"Training Accuracies: {train_accuracies}")

# Evaluate on test set
test_accuracies= evaluate_model(model_ft, testLoader, len(testDS))
print(f"Test Accuracies: {test_accuracies}")


Training Accuracies: {'ProductType': 0.9725609756097561}
Test Accuracies: {'ProductType': 0.6097560975609756}


In [16]:
all_filepaths, all_prdlabel_preds = [], []

for i in range(len(annotations_2)):
    image_path = os.path.join(CONFIGS["DATA_BASE_PATH"], annotations_2['filepath'][i])
    frame = cv2.imread(image_path)

    # Preprocessing steps
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (CONFIGS['SMALL_MODEL_IMG_SIZE'], CONFIGS['SMALL_MODEL_IMG_SIZE']))
    frame = frame.transpose((2, 0, 1))
    frame = torch.from_numpy(frame).float()
    frame = test_transforms(frame).unsqueeze(0).to(CONFIGS['DEVICE'])

    # Perform prediction
    with torch.no_grad():
        out1 = model_ft(frame)
    
    # Extract and store the results
    tmp_pred = out1.argmax(1)
    tmp_pred_label = le_prdtype.inverse_transform([tmp_pred])
    all_prdlabel_preds.append(tmp_pred_label[0])


print("Accuracy on new images")
print(sum(all_prdlabel_preds == annotations_2['ProductType']) / len(annotations_2))

Accuracy on new images
1.0


In [17]:
annotations.shape

(410, 15)

In [18]:
all_logits = []

for i in range(len(annotations)):
    image_path = os.path.join(CONFIGS["DATA_BASE_PATH"], annotations['filepath'][i])
    frame = cv2.imread(image_path)

    # Preprocessing steps
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (CONFIGS['SMALL_MODEL_IMG_SIZE'], CONFIGS['SMALL_MODEL_IMG_SIZE']))
    frame = frame.transpose((2, 0, 1))
    frame = torch.from_numpy(frame).float()
    frame = test_transforms(frame).unsqueeze(0).to(CONFIGS['DEVICE'])

    # Perform prediction
    with torch.no_grad():
        out1 = model_ft(frame)
    
    # Extract and store the results
    all_logits.append(out1[0].tolist())


In [19]:
# Convert list of lists to DataFrame
all_logits_df = pd.DataFrame(all_logits)
all_logits_df.columns = le_prdtype.classes_
all_logits_df['pred_prdtype'] = all_logits_df.idxmax(axis=1)
target_column = 'pred_prdtype'
columns = [target_column] + [col for col in all_logits_df.columns if col != target_column]
all_logits_df = all_logits_df[columns]

final_df = pd.concat([annotations, all_logits_df], axis=1)
final_df.head()

,filepath,xmin,ymin,xmax,ymax,label,ProductType,Weight,HalalStatus,HealthStatus,...,Potatochips,RiceBrownOthers,RiceWhite,RolledOatsInstantOatmeal,Salt,SoftDrinksOtherReadyToDrink,SoupStock,Sugar,SweetsChocolatesOthers,TeaPowderLeaves
0,2023_10_25_12_0_34_784872.jpg,285,168,567,528,BabyMilkPowder_200-299g_Halal_NonHealthy,BabyMilkPowder,200-299g,Halal,NonHealthy,...,-1.538441,-2.474456,-2.166450,-2.289746,-1.916843,-0.356248,-1.462294,-1.907547,-1.365697,-2.333017
1,2023_10_25_11_50_16_569076.jpg,266,207,506,528,BabyMilkPowder_400-499g_Halal_NonHealthy,BabyMilkPowder,400-499g,Halal,NonHealthy,...,-0.542177,-2.324751,-1.815467,-2.294456,-1.651892,-1.226976,-2.332330,0.209407,-1.838781,-2.299150
2,2023_10_25_11_45_37_343791.jpg,230,80,562,528,BabyMilkPowder_800-899g_Halal_Healthy,BabyMilkPowder,800-899g,Halal,Healthy,...,1.371769,-1.519539,-0.456094,0.085578,-2.225759,-1.688356,-2.762919,-0.552690,-0.507280,-1.503624
3,2023_10_25_12_9_12_144686.jpg,200,140,532,528,BabyMilkPowder_900-999g_NonHalal_NonHealthy,BabyMilkPowder,900-999g,NonHalal,NonHealthy,...,0.002006,-2.333671,-1.224976,-2.133831,-0.818596,-1.456577,-2.075854,0.011306,-1.141832,-1.817970
4,2023_10_25_12_8_11_460440.jpg,218,125,536,528,BabyMilkPowder_800-899g_Halal_NonHealthy,BabyMilkPowder,800-899g,Halal,NonHealthy,...,-0.578252,-2.372956,-2.174664,-1.919113,-2.649999,-0.679314,-1.787639,-0.066358,-0.924837,-2.095980


In [20]:
sum(final_df['ProductType'] == final_df['pred_prdtype']) / len(final_df)

0.9219512195121952

In [21]:
final_df.to_csv('new_imgs_results_small_model.csv', index=False)

In [22]:
final_df.columns

Index(['filepath', 'xmin', 'ymin', 'xmax', 'ymax', 'label', 'ProductType',
       'Weight', 'HalalStatus', 'HealthStatus', 'new_camera', 'tag', 'remove',
       'tokeep', 'type', 'pred_prdtype', 'BabyMilkPowder', 'Babyfood',
       'BeehoonVermicelli', 'BiscuitsCrackersCookies', 'BreakfastCereals',
       'CannedBakedBeans', 'CannedBeefOtherMeats', 'CannedBraisedPeanuts',
       'CannedChicken', 'CannedFruits', 'CannedMushrooms',
       'CannedPacketCreamersSweet', 'CannedPickles', 'CannedPorkLunchronMeat',
       'CannedSardinesMackerel', 'CannedSoup', 'CannedTunaDace',
       'CannedVegetarianFood', 'ChocolateMaltPowder', 'ChocolateSpread',
       'CoffeePowder', 'CoffeeTeaDrink', 'CookingCreamMilk',
       'CookingPastePowder', 'DarkSoySauce', 'DriedBeans', 'DriedFruits',
       'DriedMeatSeafood', 'DriedVegetables', 'FlavoredMilkDrink', 'Flour',
       'FruitJuiceDrink', 'HerbsSpices', 'InstantMeals',
       'InstantNoodlesMultipack', 'InstantNoodlesSingle', 'Jam', 'Kaya',
       '

In [23]:
annotations['ProductType'].unique()

array(['BabyMilkPowder', 'Babyfood', 'BeehoonVermicelli',
       'BiscuitsCrackersCookies', 'BreakfastCereals', 'CannedBakedBeans',
       'CannedBeefOtherMeats', 'CannedBraisedPeanuts', 'CannedChicken',
       'CannedFruits', 'CannedMushrooms', 'CannedPacketCreamersSweet',
       'CannedPickles', 'CannedPorkLunchronMeat',
       'CannedSardinesMackerel', 'CannedSoup', 'CannedTunaDace',
       'CannedVegetarianFood', 'ChocolateMaltPowder', 'ChocolateSpread',
       'CoffeePowder', 'CoffeeTeaDrink', 'CookingCreamMilk',
       'CookingPastePowder', 'DarkSoySauce', 'DriedBeans', 'DriedFruits',
       'DriedMeatSeafood', 'DriedVegetables', 'FlavoredMilkDrink',
       'Flour', 'FruitJuiceDrink', 'HerbsSpices', 'InstantMeals',
       'InstantNoodlesMultipack', 'InstantNoodlesSingle', 'Jam', 'Kaya',
       'KetchupChilliSauce', 'LightSoySauce', 'MaternalMilkPowder',
       'MilkDrink', 'MilkPowder', 'Nuts', 'Oil', 'OtherBakingNeeds',
       'OtherCannedBeansPeasNuts', 'OtherCannedSeafood',
  